In [1]:
from utils import *

from PyQt5.QAxContainer import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *

from datetime import datetime, timedelta
from copy import deepcopy
import pandas as pd
import numpy as np
import pickle
import random
import time
import glob
import sys
import os

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import mpl_finance
from matplotlib import font_manager, rc

ModuleNotFoundError: No module named 'PyQt5'

In [5]:
!conda list

# packages in environment at C:\Users\changyong\Anaconda3\envs\py32bit:
#
# Name                    Version                   Build  Channel
bleach                    3.3.1              pyhd3eb1b0_0  
certifi                   2018.8.24                py35_1    anaconda
colorama                  0.4.4              pyhd3eb1b0_0  
decorator                 5.0.9              pyhd3eb1b0_0  
defusedxml                0.7.1              pyhd3eb1b0_0  
entrypoints               0.2.3                    py35_2  
import-ipynb              0.1.3                    pypi_0    pypi
ipykernel                 4.10.0                   py35_0  
ipython                   5.8.0                    py35_0  
ipython_genutils          0.2.0              pyhd3eb1b0_1  
jinja2                    2.11.3             pyhd3eb1b0_0  
jsonschema                2.6.0            py35h27d56d3_0  
jupyter_client            5.3.3                      py_0  
jupyter_core              4.5.0                      py_0  
lib

In [2]:
####### 테스트용
import import_ipynb
import win32com.client
import pythoncom


import xing_login
import win32com


class XQuery_t1101:
    # classmethod get_instance() 를 사용하여, instance 를 만들어야함.
    def __init__(self):
        self.is_data_received = False

    def OnReceiveData(self, tr_code): # event handler
        # 이베스트 서버에서 ReceiveData 이벤트 받으면 실행되는 event handler
        self.is_data_received = True
        name = self.GetFieldData("t1101OutBlock", "hname", 0)
        price = self.GetFieldData("t1101OutBlock", "price", 0)
        volume = self.GetFieldData("t1101OutBlock", "volume", 0)
        print("종목; {0}".format(name))
        print("현재가; {0}".format(price))
        print("누적거래량; {0}".format(volume))

        print("tr code ==> {0}".format(tr_code))

    def single_request(self, stockcode):
        self.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res" # RES 파일 등록
        self.SetFieldData("t1101InBlock", "shcode", 0, stockcode) # 종목코드 설정
        err_code = self.Request(False) # data 요청하기 -- 연속조회인경우만 True

        if err_code < 0:
            print("error... {0}".format(err_code))

    @classmethod
    def get_instance(cls):
        # DispatchWithEvents로 instance 생성하기
        xq_t1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", cls)
        return xq_t1101


if __name__ == "__main__":
    def get_single_data():
        xq_t1101 = XQuery_t1101.get_instance()
        xq_t1101.single_request("005930") # 삼성전자.

        while xq_t1101.is_data_received == False:
            pythoncom.PumpWaitingMessages()


    xsession = xing_login.XSession.get_instance()
    xsession.api_login()

    get_single_data()

ModuleNotFoundError: No module named 'import_ipynb'

In [12]:
class Price:
    def __init__(self, filepath, filepath_sample, sample, period, keys_init):
        self.filepath = filepath
        self.filepath_sample = filepath_sample
        self.filename_sample = sample # 영업일 리스트를 불러올 샘플 데이터
        self.code_dfs = {} # 중복으로 데이터를 불러오지 않기 위해 종목코드마다 불러온 데이터를 할당.
        self.business_dates_list, self.business_dates_dict = self._getBusinessDates()
        # 시작일, 각 영업년/월/일별 인덱스값을 가진 딕셔너리 생성 (리스트의 빠른 역함수 역할)
        self.period = period
        self.keys = keys_init
        self._appendKeys()
        # dict형태, major1, 2 minor1, 2 있음
        
            
    def _findDateIndexSelf(self, date): # pkl 파일마다 시작일이 다를 수 있으므로 이를 고려해서 인덱스값 반환
        (y, m, d) = date.split('-')
        index = self.business_dates_dict[y][m][d]
        return index
        
    def _findDateIndex(self, df, date, with_self = False): # pkl 파일마다 시작일이 다를 수 있으므로 이를 고려해서 인덱스값 반환
        (y0, m0, d0) = df.index[0].split('-')
        index0 = self.business_dates_dict[y0][m0][d0]
        index = self._findDateIndexSelf(date)
        if with_self:
            return index - index0, index
        return index - index0
    
    def _getBusinessDates(self):
        with open('{}/{}.pkl'.format(self.filepath_sample, self.filename_sample), 'rb') as f:
             sample_df = pickle.load(f)
        list = sample_df.index
        dict = {}
        for i, val in enumerate(sample_df.index):
            if i == 0:
                dict['start_date'] = val
            (y, m, d) = val.split('-')
            if not y in dict.keys():
                dict[y] = {}
            if not m in dict[y].keys():
                dict[y][m] = {}
            dict[y][m][d] = i
        return list, dict
    
    def _getDf(self, code):
        if not code in self.code_dfs.keys():
            with open('{}/{}.pkl'.format(self.filepath, code), 'rb') as f:
                df = pickle.load(f)
            self.code_dfs[code] = df
            log('{}.pkl is opened'.format(code))
            return df
        else:
            return self.code_dfs[code]
    
    def getPrices(self, code, date_start, period, value_type = 'close', with_df = False): # start date을 포함해서 period만큼의 데이터를 리스트로 출력
        df = self._getDf(code)
#         index_start, index_start_self = self._findDateIndex(df, date_start, with_self = True)
#         assert self.business_dates_list[index_start_self+period-1] == df.index[index_start+period-1], 'ErrorLog : business dates matching error'
        index_start = list(df.index).index(date_start)
        if with_df:
            return df[value_type][index_start:index_start+period], df
        return [*df[value_type][index_start:index_start+period]]
        
    def getInterests(self, code, date_base, date_start, period, value_type = 'close'):
        price_list, df = self.getPrices(code, date_start, period, value_type, with_df = True)
        price_base = df[value_type][self._findDateIndex(df, date_base)]
        return [(price / price_base) - 1 for price in price_list]
    
    def nextDate(self, date):
        return self.business_dates_list[self._findDateIndexSelf(date) + 1]
    
    def _appendKeys(self):
        for i in range(self.period):
            self.keys['minor1'].append(i+1)
            self.keys['minor2'].append(i+1)
            if i == 0:
                self.keys['major1'].append('{} 영업일 수익률'.format(self.period))
                self.keys['major2'].append('{} 영업일 수익률'.format(self.period))
            else:
                self.keys['major1'].append('')
                self.keys['major2'].append('')
    
    def appendData(self, code, date_base, date_start, line1, line2):
        line1 = [*line1]
        line2 = [*line2]
        interest_list = self.getInterests(code, date_base, date_start, self.period)
        line1.extend(interest_list)
        line2.extend(interest_list)
        return line1, line2
    
# 로그용 함수
def log(content):
    print('Log : {}'.format(content))

In [ ]:
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

abs_path = os.path.realpath(__file__).split('\\')[:-2]
abs_path = '/'.join(abs_path)

TR_REQ_TIME_INTERVAL = 1.0 #0.2
SCREEN_NO = '1000'

In [3]:
class PriceAgent:
    def __init__(self, window):
        self.today_date = datetime.strftime(datetime.now(),"%Y%m%d")
        self.window = window
        self.code_list = []
        self.save_list = []
        self.data_num = None
        self.day0_date = None
        if os.path.exists('day_before.pkl'):
            with open('day_before.pkl', 'rb') as f:
                self.day_before, is_complete = pickle.load(f)
            if is_complete:
                self.day_before -= 1
        else:
            self.day_before = 23
        assert self.day_before >= 2

    def event_connect(self, kiwoom):
        #################### 조건검색하기 #######################
        kiwoom.condition_name = '5선이격조건{}일봉전'.format(self.day_before)
        kiwoom.get_condition_load()
        loop = QEventLoop()
        kiwoom.signal.finished.connect(loop.quit)
        loop.exec_()
        self.code_list = deepcopy(kiwoom.cond_code_list)

        #################### 일봉데이터 가져오기 #######################
        self.daily_data_list = []
        for code in self.code_list:
            kiwoom.get_daily_price_data(code, self.today_date)
            loop = QEventLoop()
            kiwoom.signal.finished.connect(loop.quit)
            loop.exec_()

            daily_data = deepcopy(kiwoom.daily_data)
            if code in kiwoom.stockCodeList_KOSPI:
                market_type = 'KOSPI'
            elif code in kiwoom.stockCodeList_KOSDAQ:
                market_type = 'KOSDAQ'
            else:
                market_type = 'ETC'
            self.daily_data_list.append([code, daily_data, market_type])
        self.data_num = len(self.daily_data_list)
        print("조건식 만족 코드 가져오기 성공! 총 {}개".format(self.data_num))

        if self.data_num == 0:
            print("조건식 만족 코드 없음 종료")
            with open('day_before.pkl', 'wb') as f:
                pickle.dump([self.day_before, True], f)
            kiwoom.program_exit()
            return

        date_list = self.daily_data_list[0][1].index
        self.day0_date = date_list[-1-self.day_before]

        if self.day_before > 2:
            remove_rows = date_list[2-self.day_before:]
            for i, daily_data in enumerate(self.daily_data_list):
                self.daily_data_list[i][1].drop(remove_rows, axis=0, inplace=True)

        #################### 일봉데이터 유무확인 #######################
        if not os.path.isdir('{}/data/{}'.format(abs_path, self.day0_date)):
            os.mkdir('{}/data/{}'.format(abs_path, self.day0_date))
        with open('{}/data/{}/daily_data.pkl'.format(abs_path, self.day0_date), 'wb') as f:
            pickle.dump(self.daily_data_list, f)

        #time.sleep(60 + random.random()*2)

        #####################################아래부터는 분봉데이터###################################

        minute_data_dir = '{}/data/minute_data'.format(abs_path)
        if not os.path.isdir(minute_data_dir):
            os.mkdir(minute_data_dir)

        for idx in range(len(self.daily_data_list)):
            jongmok_code, daily_data, market_type = self.daily_data_list[idx]
            original_data = []
            if '{}.pkl'.format(jongmok_code) in os.listdir('{}/'.format(minute_data_dir)):
                print('[{}] 분봉데이터 있음'.format(jongmok_code))
                with open('{}/{}.pkl'.format(minute_data_dir, jongmok_code), 'rb') as f:
                    original_data = pickle.load(f)
                #continue
            print('[{}] 분봉데이터 긁어오기 시작!'.format(jongmok_code))
            kiwoom.min_data = {'open':[], 'high':[], 'low':[], 'close':[], 'volume':[]}
            kiwoom.min_date = []

            next = 0
            while True:
                if kiwoom.cnt >= 999:
                    print("과도한 데이터 요청 방지 프로그램 종료")
                    with open('day_before.pkl', 'wb') as f:
                        pickle.dump([self.day_before, False], f)
                    kiwoom.program_exit()
                    return

                loop = QEventLoop()
                kiwoom.signal.finished.connect(loop.quit)
                kiwoom.get_min_price_data(jongmok_code, next)
                loop.exec_()
                next = kiwoom.miniute_next
                if next == 0:
                    break
                if len(original_data) != 0:
                    if kiwoom.min_date[-1] in original_data.index:
                        break

            kiwoom.min_date = kiwoom.min_date[::-1]
            for col in ['open','high','low','close','volume']:
                kiwoom.min_data[col] = kiwoom.min_data[col][::-1]
            data = pd.DataFrame(kiwoom.min_data, columns=['open', 'high', 'low', 'close', 'volume'], index=kiwoom.min_date)

            if len(original_data) != 0:
                data = data.loc[original_data.index[-1]:]
                if len(data) > 1:
                    data = data.loc[data.index[1]:]
                    data = original_data.append(data)
                else:
                    data = original_data
            
            with open('{}/{}.pkl'.format(minute_data_dir, jongmok_code), 'wb') as f:
                pickle.dump(data, f)
            print(datetime.strptime(kiwoom.min_date[0],'%Y%m%d%H%M%S'), "분봉데이터 조회 끝!!!!")
            #time.sleep(60 + random.random()*2)

        print("test data 긁어오기 종료")
        with open('day_before.pkl', 'wb') as f:
            pickle.dump([self.day_before, True], f)
        kiwoom.program_exit()
        return

#######################################################################################################

class MyWindow(QWidget):
    def __init__(self, app):
        super().__init__()
        self.app = app
        self.setupUI()
        self.agent = PriceAgent(self)
        self.kiwoom = Kiwoom(self)

    def event_connect(self):
        self.agent.event_connect(self.kiwoom)

    def setupUI(self):
        self.setGeometry(100, 100, 100, 100)
        self.setWindowTitle("dobro")
        layout = QHBoxLayout()
        self.setLayout(layout)
        

class Custom_Signal(QWidget):
    #시그널 만들기
    finished = pyqtSignal(int)
    total_finished = pyqtSignal(int)
    def __init__(self):
        super().__init__()


class Kiwoom(QAxWidget):
    def __init__(self, window):
        super().__init__("KHOPENAPI.KHOpenAPICtrl.1")

        self.window = window
        self.signal = Custom_Signal()
        self.condition_name = None
        self.cnt = 0

        self.OnEventConnect.connect(self._event_connect)
        self.OnReceiveTrData.connect(self._receive_tr_data)
        self.OnReceiveRealData.connect(self._receive_real_data)
        self.OnReceiveMsg.connect(self._receive_msg)
        self.OnReceiveChejanData.connect(self._receive_chejan_data)
        self.OnReceiveConditionVer.connect(self._receive_condition_ver)
        self.OnReceiveTrCondition.connect(self._receive_tr_condition)

        self.signal.total_finished.connect(self.window.app.quit)
        
        self.comm_connect()

    def program_exit(self):
        print("프로그램 종료!")
        self.signal.total_finished.emit(0)

    def comm_connect(self):
        self.dynamicCall("CommConnect()")

    def set_input_value(self, id, value):
        self.dynamicCall("SetInputValue(QString, QString)", id, value)

    def comm_rq_data(self, rqname, trcode, next, screen_no):
        err = self.dynamicCall("CommRqData(QString, QString, int, QString)", rqname, trcode, next, screen_no)
        if err != 0:
            raise ValueError("[{} 요청]에서 에러 발생! 에러코드 : [{}]".format(rqname, err))
        self.cnt += 1
        print(self.cnt)
        time.sleep(TR_REQ_TIME_INTERVAL)

    def _get_comm_data(self, code, request_name, index, item_name):
        ret = self.dynamicCall("GetCommData(QString, QString, int, QString)", code, request_name, index, item_name)
        return ret.strip()

    def _get_comm_real_data(self, code, index):
        ret = self.dynamicCall("GetCommRealData(QString, int)", code, index)
        return ret.strip()

    def _get_chejan_data(self, fid):
        ret = self.dynamicCall("GetChejanData(int)", fid)
        return ret.strip()

    def get_daily_price_data(self, jongmok_code, end_day="20170224"):
        rqname = "일봉차트조회"
        trcode = "opt10081"
        screen_no = SCREEN_NO
        self.set_input_value("종목코드", jongmok_code)
        self.set_input_value("기준일자", end_day)
        self.set_input_value("수정주가구분", 1)
        self.comm_rq_data(rqname, trcode, 0, screen_no)
        print('[{} 요청] 종목코드 : {}'.format(rqname, jongmok_code))

    def get_min_price_data(self, jongmok_code, next=0):
        rqname = "분봉차트조회"
        trcode = "opt10080"
        screen_no = SCREEN_NO
        self.set_input_value("종목코드", jongmok_code)
        self.set_input_value("틱범위", 1)
        self.set_input_value("수정주가구분", 1)
        self.comm_rq_data(rqname, trcode, next, screen_no)
        #print('[{} 요청] 종목코드 : {}'.format(rqname, jongmok_code))

    def _event_connect(self, err_code):
        if err_code == 0:
            print("connected")

            self.stockCodeList_KOSPI = self.dynamicCall("GetCodeListByMarket(QString)", "0")
            self.stockCodeList_KOSPI = self.stockCodeList_KOSPI.split(';')[:-1]
            self.stockCodeList_KOSDAQ = self.dynamicCall("GetCodeListByMarket(QString)", "10")
            self.stockCodeList_KOSDAQ = self.stockCodeList_KOSDAQ.split(';')[:-1]
            account_list = self.dynamicCall("GetLoginInfo(QString)","ACCLIST")
            account_list = account_list.split(';')[:-1]
            
            self.window.event_connect()

        else:
            print("disconnected")

    def _receive_tr_data(self, screen_no, rqname, trcode, record_name, next, unused1, unused2, unused3, unused4):
        if rqname == "일봉차트조회":
            cnt = self.dynamicCall("GetRepeatCnt(QString, QString)", trcode, rqname)
            jongmok_code = self._get_comm_data(trcode, rqname, 0, "종목코드")
            data = {'open':[], 'high':[], 'low':[], 'close':[], 'volume':[]}
            date = []
            for i in range(cnt-1, -1, -1):
                d = self._get_comm_data(trcode, rqname, i, "일자")
                date.append(d[:4]+'-'+d[4:6]+'-'+d[6:])
                data['open'].append(abs(int(self._get_comm_data(trcode, rqname, i, "시가"))))
                data['high'].append(abs(int(self._get_comm_data(trcode, rqname, i, "고가"))))
                data['low'].append(abs(int(self._get_comm_data(trcode, rqname, i, "저가"))))
                data['close'].append(abs(int(self._get_comm_data(trcode, rqname, i, "현재가"))))
                data['volume'].append(abs(int(self._get_comm_data(trcode, rqname, i, "거래량"))))
            data = pd.DataFrame(data, columns=['open', 'high', 'low', 'close', 'volume'], index=date)
            self.daily_data = data
            print('[{}] 종목코드 : {} 수신성공'.format(rqname,jongmok_code))
            self.signal.finished.emit(0)

        elif rqname == "분봉차트조회":
            cnt = self.dynamicCall("GetRepeatCnt(QString, QString)", trcode, rqname)
            jongmok_code = self._get_comm_data(trcode, rqname, 0, "종목코드")
            for i in range(cnt):
                self.min_date.append(self._get_comm_data(trcode, rqname, i, "체결시간"))
                self.min_data['open'].append(abs(int(self._get_comm_data(trcode, rqname, i, "시가"))))
                self.min_data['high'].append(abs(int(self._get_comm_data(trcode, rqname, i, "고가"))))
                self.min_data['low'].append(abs(int(self._get_comm_data(trcode, rqname, i, "저가"))))
                self.min_data['close'].append(abs(int(self._get_comm_data(trcode, rqname, i, "현재가"))))
                self.min_data['volume'].append(abs(int(self._get_comm_data(trcode, rqname, i, "거래량"))))
            self.miniute_next = int(next)
            self.signal.finished.emit(0)
            '''
            if next == "2":
                print(self.min_date[-1])
                self.get_min_price_data(jongmok_code, 2)
            else:
                self.min_date = self.min_date[::-1]
                for col in ['open','high','low','close','volume']:
                    self.min_data[col] = self.min_data[col][::-1]
                data = pd.DataFrame(self.min_data, columns=['open', 'high', 'low', 'close', 'volume'], index=self.min_date)
                with open(abs_path+'data/{}/minute_data_{}.pkl'.format(self.cond_date, jongmok_code), 'wb') as f:
                    pickle.dump(data, f)
                print(datetime.strptime(self.min_date[0],'%Y%m%d%H%M%S'), "분봉데이터 조회 끝!!!!")
                self.signal.finished.emit(0)
            '''


    def _receive_real_data(self, jongmok_code, real_type, real_data):
        pass

    def _receive_msg(self, screen_no, rqname, trcode, msg):
        #msg : [00z310] 모의투자 조회, [00z112] 매수 정상처리, [00z113] 매도 정상처리, [00z214] 매수 주문수량 오류
        print("[OnReceiveMsg] screen : {}, rqname : {}, trcode : {}, msg : {}".format(screen_no,rqname,trcode,msg))

    def _receive_chejan_data(self, gubun, item_cnt, fid_list):
        pass

    ##########################
    ######## 조건 검색 ########
    ##########################
    def get_condition_load(self):
        e = self.dynamicCall("GetConditionLoad()")
        if e == 0:
            raise ValueError("[조건검색 요청]에서 에러 발생!")
        time.sleep(TR_REQ_TIME_INTERVAL)

    def get_condition_name_list(self):
        value = self.dynamicCall("GetConditionNameList()")
        return value

    def send_condition(self, screen_no, cond_name, cond_idx, cond_type=0):
        #cond_type = 0:조건검색, 1:실시간 조건검색
        e = self.dynamicCall("SendCondition(QString, QString, int, int)", screen_no, cond_name, cond_idx, cond_type)
        if e == 0:
            raise ValueError("[조건종목 요청]에서 에러 발생!")
        time.sleep(TR_REQ_TIME_INTERVAL)

    def _receive_condition_ver(self, ret, msg):
        print("[조건검색] return : {}, msg : {}".format(ret, msg))
        value = self.get_condition_name_list().split(';')
        condition_list = []
        for item in value:
            condition = item.split('^')
            if len(condition) == 2 and condition[1] == self.condition_name:
                condition_list = condition
                break
        
        if len(condition_list) == 0:
            raise ValueError('[{}]을 만족하는 조건식이 없음!'.format(self.condition_name))
        
        print('[조건검색결과] {}'.format(condition_list))
        self.send_condition(SCREEN_NO, condition_list[1], condition_list[0])

    def _receive_tr_condition(self, screen_no, code_list, cond_name, idx, next):
        print("[조건종목] 데이터 도착 | screen : {}, cond_name : {}, idx : {}, next : {}".format(screen_no, cond_name, idx, next))
        code_list = code_list.split(';')
        if len(code_list) > 0:
            code_list = code_list[:-1]
        self.cond_code_list = code_list
        self.signal.finished.emit(0)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MyWindow(app)
    window.show()
    app.exec_()
        

AttributeError: 'Kiwoom' object has no attribute 'OnEventConnect'

In [ ]:
if __name__ == '__main__':
    filepath = '../kw/prices/all_daily_data_modified_close'
    filename_sample = '000020'
    with_price = Price(filepath, filepath, filename_sample, 120, {'major1':[], 'minor1':[], 'major2':[], 'minor2':[]})
#     print(with_price.business_dates)
    with open(filepath+'/000020.pkl', 'rb') as f:
        df = pickle.load(f)
    print(df)
    print(df['low'][4:100])
    print(len(df.index))
    print(df.index)
    print(with_price._findDateIndex(df['open'][4:100], '2016-06-27'))
    print(with_price.getPrices('000060', '2021-04-26', 3))
    print(with_price.getPrices('000060', '2021-04-26', 3))
    print(with_price.getInterests('000500', '2019-04-26', '2019-04-26', 120))
    print(with_price.nextDate('2019-04-26'))

In [16]:
# # Price 데이터 경로 입력
# filepath_price = '../kw/prices/all_daily_data_modified_close'
# filepath_price_sample = '../kw/prices/all_daily_data'
# filename_price_sample = '000020' # 영업일 리스트를 불러올 샘플 데이터

# monitoring_period = 120

# keys_init = {'major1': [], 'major2': [], 'minor1': [], 'minor2': []}
# with_price = Price(filepath_price, filepath_price_sample, filename_price_sample, monitoring_period, keys_init)
# print(with_price.nextDate('2018-01-02'))
# print(with_price.appendData('001070', '2018-01-02', with_price.nextDate('2018-01-02'), [], []))

2018-01-03
Log : 001070.pkl is opened
2016 05 17
2016 05 17
([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0608974358974359, -0.06730769230769229, -0.08974358974358976, -0.08974358974358976, -0.10897435897435892, -0.10897435897435892, -0.11217948717948723, -0.10897435897435892, -0.10256410256410253, -0.10576923076923073, -0.11217948717948723, -0.11858974358974361, -0.1314102564102564, -0.15384615384615385, -0.16025641025641024, -0.15705128205128205, -0.15705128205128205, -0.15384615384615385, -0.15705128205128205, -0.15705128205128205, -0.16346153846153844, -0.16666666666666663, -0.16346153846153844, -0.16346153846153844, -0.16987179487179482, -0.17628205128205132, -0.17628205128205132, -0.17948717948717952, -0.1826923076923077, -0.20192307692307687, -0.20512820512820518, -0.22435897435897434, -0.22435897435897434, -0.21474358974358976, -0.19871794871794868, -0.19871794871794868, -0.19871794871794868, -0.19871794871794868, -0.19551282051282048, -0.19551282051282048, -0.189

In [11]:
# print(with_price.business_dates_dict['2016'])

{'06': {'15': 0, '16': 1, '17': 2, '20': 3, '21': 4, '22': 5, '23': 6, '24': 7, '27': 8, '28': 9, '29': 10, '30': 11}, '07': {'01': 12, '04': 13, '05': 14, '06': 15, '07': 16, '08': 17, '11': 18, '12': 19, '13': 20, '14': 21, '15': 22, '18': 23, '19': 24, '20': 25, '21': 26, '22': 27, '25': 28, '26': 29, '27': 30, '28': 31, '29': 32}, '08': {'01': 33, '02': 34, '03': 35, '04': 36, '05': 37, '08': 38, '09': 39, '10': 40, '11': 41, '12': 42, '16': 43, '17': 44, '18': 45, '19': 46, '22': 47, '23': 48, '24': 49, '25': 50, '26': 51, '29': 52, '30': 53, '31': 54}, '09': {'01': 55, '02': 56, '05': 57, '06': 58, '07': 59, '08': 60, '09': 61, '12': 62, '13': 63, '19': 64, '20': 65, '21': 66, '22': 67, '23': 68, '26': 69, '27': 70, '28': 71, '29': 72, '30': 73}, '10': {'04': 74, '05': 75, '06': 76, '07': 77, '10': 78, '11': 79, '12': 80, '13': 81, '14': 82, '17': 83, '18': 84, '19': 85, '20': 86, '21': 87, '24': 88, '25': 89, '26': 90, '27': 91, '28': 92, '31': 93}, '11': {'01': 94, '02': 95, '0